# Install libraries

In [ ]:
! pip install num2words

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.3/143.3 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=d85b199e63703edc4936a62be3d6c0f73ec792c9a9b6730ac8f7ed89336ddc10
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


# Import libraries

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import re
import nltk
import nltk.stem
from num2words import num2words

# NLTK for natural language processing tasks
from nltk.tokenize import word_tokenize, sent_tokenize, PunktSentenceTokenizer
from nltk import pos_tag, sent_tokenize, word_tokenize, BigramAssocMeasures,\
    BigramCollocationFinder, TrigramAssocMeasures, TrigramCollocationFinder
from nltk.corpus import stopwords
from string import punctuation

# Visualization libraries
from plotly import tools
import plotly as py
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
import plotly.graph_objs as go

import sys
import warnings

# Ignore warnings
if not sys.warnoptions:
    warnings.simplefilter("ignore")

# Matplotlib and Seaborn for data visualization
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

# Scikit-learn for machine learning tasks
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.feature_extraction.text import TfidfVectorizer

# TensorFlow and Keras for deep learning tasks
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow import keras
from tensorflow.keras import layers

# Download NLTK resources
nltk.download('omw-1.4')

# Download spaCy model
from nltk.corpus import stopwords
import nltk
from nltk.stem import WordNetLemmatizer, PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
import unicodedata
from num2words import num2words
import time

# Mount Google Drive (assuming Colab environment)
from google.colab import drive
drive.mount('/content/drive')



[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Mount Drive and Read the file
data = pd.read_csv(f'/content/drive/My Drive/Master Notebook/un-general-debates.csv')
data.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,session,year,country,text
0,44,1989,MDV,﻿It is indeed a pleasure for me and the member...
1,44,1989,FIN,"﻿\nMay I begin by congratulating you. Sir, on ..."
2,44,1989,NER,"﻿\nMr. President, it is a particular pleasure ..."
3,44,1989,URY,﻿\nDuring the debate at the fortieth session o...
4,44,1989,ZWE,﻿I should like at the outset to express my del...


In [ ]:
# Check for duplicate data
print("Number of duplicate rows before preprocessing:", data.duplicated().sum())

# Remove duplicate data
data.drop_duplicates(inplace=True)

# Check for missing data
print("Number of missing values before preprocessing:")
print(data.isnull().sum())


Number of duplicate rows before preprocessing: 0
Number of missing values before preprocessing:
session    0
year       0
country    0
text       0
dtype: int64


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7507 entries, 0 to 7506
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   session  7507 non-null   int64 
 1   year     7507 non-null   int64 
 2   country  7507 non-null   object
 3   text     7507 non-null   object
dtypes: int64(2), object(2)
memory usage: 293.2+ KB


Data Pre-processing: Pre-process the dataset to remove any artifacts from OCR scans, handle missing values (if any) and erform any necessary text cleaning and normalization.

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

# Function for lemmatization
def lemmatize_text(text):
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(word) for word in text.split()]
    return ' '.join(lemmatized_tokens)

# Function for stemming
def stem_text(text):
    stemmer = PorterStemmer()
    stemmed_tokens = [stemmer.stem(word) for word in text.split()]
    return ' '.join(stemmed_tokens)

# Function for vectorization
def vectorize_text(text):
    vectorizer = TfidfVectorizer()
    text_features = vectorizer.fit_transform(text)
    return text_features

# Function for preprocess the text
def preprocess_text(text, lower_case=True, convert_numbers=True, remove_punctuation=True,
                    remove_accents=True, remove_whitespace=True, expand_abbreviations=False,
                    remove_stopwords=False, sparse_terms=[], specific_words=[]):

    # Convert all letters to lower or upper case
    if lower_case:
        text = text.lower()
    else:
        text = text.upper()

    # Converting numbers into words or removing numbers
    if convert_numbers:
        # Replace numbers with their word equivalents using num2words library
        tokens = []
        for word in text.split():
            if word.isdigit():
                word = num2words(int(word))
            tokens.append(word)
        text = ' '.join(tokens)
    else:
        # Remove numbers using regular expression
        text = re.sub(r'\d+', '', text)

    # Removing punctuations, accent marks and other diacritics
    if remove_punctuation:
        text = re.sub(r'[^\w\s]', '', text)
    if remove_accents:
        text = ''.join(c for c in unicodedata.normalize('NFD', text)
                       if not unicodedata.combining(c))

    # Removing white spaces
    if remove_whitespace:
        text = re.sub(r'\s+', ' ', text.strip())

    # Expanding abbreviations
    if expand_abbreviations:
        abbreviations = {'e.g.': 'for example', 'i.e.': 'that is', 'etc.': 'et cetera'}
        for abbr, expanded in abbreviations.items():
            text = text.replace(abbr, expanded)

    # Removing stop words, sparse terms and specific words
    if remove_stopwords:
        stop_words = set(stopwords.words('english'))
        text = ' '.join(word for word in text.split() if word not in stop_words)
    text = ' '.join(word for word in text.split() if word not in sparse_terms)
    text = ' '.join(word for word in text.split() if word not in specific_words)

    return text


# Apply the preprocessing function to a specific column
text_col = 'text'  # Specify the text column to preprocess
data['text_new'] = data[text_col].apply(preprocess_text)

# Apply lemmatization to a specific column
text_col = 'text_new'  # Specify the text column to preprocess
data[text_col] = data[text_col].apply(lemmatize_text)

# Check the preprocessed dataset and vectorized features
display(data.head())

# display(text_features.toarray())
start_time = time.time()
# # Save the preprocessed dataset
data.to_csv('preprocessed_Dataset.csv', index=False)

end_time = time.time() - start_time
print("Time taken for text preprocessing: ", end_time)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


,session,year,country,text,text_new
0,44,1989,MDV,﻿It is indeed a pleasure for me and the member...,it is indeed a pleasure for me and the member ...
1,44,1989,FIN,"﻿\nMay I begin by congratulating you. Sir, on ...",may i begin by congratulating you sir on your ...
2,44,1989,NER,"﻿\nMr. President, it is a particular pleasure ...",mr president it is a particular pleasure for m...
3,44,1989,URY,﻿\nDuring the debate at the fortieth session o...,during the debate at the fortieth session of t...
4,44,1989,ZWE,﻿I should like at the outset to express my del...,i should like at the outset to express my dele...


Time taken for text preprocessing:  7.929839372634888


# Chatbot Dataset Design:
Design the chatbot dataset in a format suitable for training and evaluating an NLP-based chatbot model. Decide on the structure of the dataset and how to organize it for Chatbot NLP tasks.

In [ ]:
# Mount Drive and Read the Pre-processed file
df = pd.read_csv(f'/content/drive/My Drive/Master Notebook/preprocessed_Dataset.csv')
df = df[['session', 'year', 'country', 'text_new']]
df.head()


,session,year,country,text_new
0,44,1989,MDV,indeed pleasure member delegation extend ambas...
1,44,1989,FIN,may begin congratulating sir election presiden...
2,44,1989,NER,mr president particular pleasure behalf delega...
3,44,1989,URY,debate fortieth session general assembly four ...
4,44,1989,ZWE,like outset express delegation satisfaction pl...


In [ ]:
import pandas as pd

# Sample user queries or prompts
user_queries = [
    "What did [country] say in [year]?",
    "Tell me about [country]'s statement in [year].",
    "What were the major issues discussed in the UN General Assembly in [year]?",
    "Give me information on [country]'s perspective in [year].",
    "What was the focus of the General Debate in session [session]?",
    "Summarize the key points from [country]'s speech in [year].",
    "Provide insights into the UN General Assembly discussions in [year].",
    "What topics did [country] emphasize during the General Debate in [year]?",
    "Describe the main themes covered in the UN General Assembly in [year].",
    "Tell me about the diplomatic priorities of [country] in [year].",
    "What issues were highlighted by [country] during the General Debate in [year]?",
    "Give me an overview of [country]'s stance on global issues in [year].",
    "What challenges did [country] address in their statement during the General Debate in [year]?",
    "Highlight the major concerns expressed by [country] at the UN General Assembly in [year].",
    "Summarize the key takeaways from the General Debate in session [session].",
]

# Create a DataFrame with user_input and model_output
chatbot_df = pd.DataFrame(columns=['session', 'year', 'country', 'text_new', 'user_input', 'model_output'])
# Iterate through the original DataFrame and sample user queries
for index, row in df.iterrows():
    for query in user_queries:
        user_input = query.replace("[country]", row['country']).replace("[year]", str(row['year'])).replace("[session]", str(row['session']))
        model_output = row['text_new']
        chatbot_df = chatbot_df.append({
            'session': row['session'],
            'year': row['year'],
            'country': row['country'],
            'text_new': row['text_new'],
            'user_input': user_input,
            'model_output': model_output
        }, ignore_index=True)

# Save the extended DataFrame to a CSV file
chatbot_df.to_csv('chatbot_dataset.csv', index=False)
chatbot_df

,session,year,country,text_new,user_input,model_output
0,44,1989,MDV,indeed pleasure member delegation extend ambas...,What did MDV say in 1989?,indeed pleasure member delegation extend ambas...
1,44,1989,MDV,indeed pleasure member delegation extend ambas...,Tell me about MDV's statement in 1989.,indeed pleasure member delegation extend ambas...
2,44,1989,MDV,indeed pleasure member delegation extend ambas...,What were the major issues discussed in the UN...,indeed pleasure member delegation extend ambas...
3,44,1989,FIN,may begin congratulating sir election presiden...,What did FIN say in 1989?,may begin congratulating sir election presiden...
4,44,1989,FIN,may begin congratulating sir election presiden...,Tell me about FIN's statement in 1989.,may begin congratulating sir election presiden...
...,...,...,...,...,...,...
22516,56,2001,HUN,first may congratulate mr han seungsoo electio...,Tell me about HUN's statement in 2001.,first may congratulate mr han seungsoo electio...
22517,56,2001,HUN,first may congratulate mr han seungsoo electio...,What were the major issues discussed in the UN...,first may congratulate mr han seungsoo electio...
22518,56,2001,KWT,behalf state kuwait give pleasure congratulate...,What did KWT say in 2001?,behalf state kuwait give pleasure congratulate...
22519,56,2001,KWT,behalf state kuwait give pleasure congratulate...,Tell me about KWT's statement in 2001.,behalf state kuwait give pleasure congratulate...


# Data Splitting:
Split the dataset into training, validation and testing sets to allow for model training and evaluation. Consider the distribution of statements across sessions and years to ensure representative splits.

In [ ]:
from sklearn.model_selection import train_test_split

# Assuming df is your DataFrame

# Perform stratified splitting based on 'session' and 'year'
train_df, temp_df = train_test_split(chatbot_df, test_size=0.3, stratify=chatbot_df[['session', 'year']], random_state=42)

# Further split the temp_df into validation and test sets
validation_df, test_df = train_test_split(temp_df, test_size=0.5, stratify=temp_df[['session', 'year']], random_state=42)

# Display the shapes of the resulting datasets
print("Training set shape:", train_df.shape)
print("Validation set shape:", validation_df.shape)
print("Testing set shape:", test_df.shape)

# Save the split datasets to CSV files
train_df.to_csv('train_dataset.csv', index=False)
validation_df.to_csv('validation_dataset.csv', index=False)
test_df.to_csv('test_dataset.csv', index=False)

train_df

Training set shape: (15764, 6)
Validation set shape: (3378, 6)
Testing set shape: (3379, 6)


,session,year,country,text_new,user_input,model_output
15918,29,1974,COL,first word addressed mr president congratulate...,What did COL say in 1974?,first word addressed mr president congratulate...
1888,63,2008,PRT,mr president wish success tenure office guaran...,Tell me about PRT's statement in 2008.,mr president wish success tenure office guaran...
3709,57,2002,CYP,would like sir extend warmest congratulation e...,Tell me about CYP's statement in 2002.,would like sir extend warmest congratulation e...
2293,46,1991,DZA,pleasure honour extend mr president congratula...,Tell me about DZA's statement in 1991.,pleasure honour extend mr president congratula...
21057,61,2006,KEN,let add voice congratulation president sheikha...,What did KEN say in 2006?,let add voice congratulation president sheikha...
...,...,...,...,...,...,...
16731,48,1993,TJK,year elapsed since forty seventh session gener...,What did TJK say in 1993?,year elapsed since forty seventh session gener...
15629,66,2011,HND,honduras source great satisfaction participate...,What were the major issues discussed in the UN...,honduras source great satisfaction participate...
1899,63,2008,CAF,first sincerely congratulate mr miguel descoto...,What did CAF say in 2008?,first sincerely congratulate mr miguel descoto...
9787,38,1983,MEX,mr president offer warmest congratulation elec...,Tell me about MEX's statement in 1983.,mr president offer warmest congratulation elec...


# Architecture and Framework Selection:
Choose an appropriate NLP framework and architecture for building the chatbot. Justify your choice based on factors such as model performance, ease of implementation and available resources.

Given the limitations related to factors such as the requirement for secret keys and GPU processing in OpenAI's GPT models, a suitable alternative for building a customized chatbot would be to use a rule-based approach with a simple NLP framework.

# Train chatbot:
Train the chatbot model on the prepared dataset and evaluate its performance using relevant NLP metrics. Discuss the model's strengths and potential limitations.

In [ ]:
import pandas as pd

chatbot_df = pd.read_csv('train_dataset.csv')

# Sample chatbot logic
def chatbot_response(user_input):
    # Find the matching row based on user input
    matching_row = chatbot_df[chatbot_df['user_input'] == user_input]

    # Check if a match is found
    if not matching_row.empty:
        return matching_row.iloc[0]['model_output']
    else:
        return "Sorry, I couldn't find information for that query."

# Simulate a conversation
user_queries = [
    "What did MDV say in 1989?",
    "What did MDV say in 1989?",

    "Tell me about FIN's statement in 1989.",
    "What were the major issues discussed in the UN General Assembly in 1989?",
]

for user_query in user_queries:
    print(f"User: {user_query}")
    model_output = chatbot_response(user_query)
    print(f"Chatbot: {model_output}\n")




User: What did MDV say in 1989?
Chatbot: indeed pleasure member delegation extend ambassador garba sincere congratulation election presidency fortyfourth session general assembly election high office welldeserved tribute personal quality experience fully confident able wise leadership assembly consolidate gain achieved past year delegation associate previous speaker expressing appreciation dedicated effort predecessor excellency mr dante caputo exemplary manner discharged duty president fortythird session general assembly previous year delegation wish note satisfaction gratitude assiduous unrelenting effort exerted secretarygeneral united nation cause peace international harmony pay tribute untiring effort promote condition conducive realization noble principle enshrined charter united nation praise congratulate success organization achieved recent year particularly praise renewed faith regeneration confidence organization ability play instrumental role peaceful settlement dispute toda

# Chatbot Deployment Considerations:
Discuss the challenges and considerations involved in deploying the trained chatbot in a real-world scenario, including integration with existing systems and user interaction design.

* Scalability: Plan for scalable server infrastructure and implement load balancing for varying levels of user interactions.

* User Interaction Design: Design a user-friendly interface, support multi-platform deployment and incorporate personalization features.

* Natural Language Understanding (NLU): Implement continuous training for NLU improvement and enhance intent recognition capabilities.

* Data Privacy and Compliance: Prioritize data encryption, comply with data protection regulations and ensure legal and privacy compliance.

* Monitoring and Analytics: Implement tools to monitor performance, track user engagement and quickly address errors.

* User Feedback and Improvement: Incorporate feedback mechanisms for continuous improvement and plan for iterative development.

* Cultural Sensitivity and Bias: Detect and mitigate biases, ensure cultural sensitivity and regularly review and update training data.

* User Authentication and Authorization: Implement secure authentication and proper authorization levels, especially for sensitive information.

* Regulatory Compliance: Conduct legal reviews, communicate terms of service and privacy policies and ensure adherence to legal requirements.

* Maintenance and Support: Establish processes for bug fixes, regular updates and user support to ensure ongoing functionality.

* Performance Testing: Conduct load and security testing to ensure optimal performance and identify vulnerabilities.

* Cross-Functional Collaboration: Collaborate with IT, DevOps and UX teams to facilitate a successful deployment and enhance user experience.

# Future Enhancements:
Propose potential future enhancements or features that could be added to improve the chatbot's functionality and performance.

While the incorporation of OpenAI's GPT models might necessitate secret keys and GPU processing, these enhancements aim to make the chatbot more sophisticated, user-friendly and adaptable over time. The implementation choices will depend on the specific goals, resources, and technical capabilities of the chatbot project.

On the other hand, the existing chatbot implementation relies on a customized approach employing rule-based methods. However, there are several potential future enhancements and features that could significantly improve its functionality and performance:

* Integration with Advanced NLP Models:
Incorporate pre-trained language models like GPT-3 for more advanced natural language understanding and generation. These models can handle long-chain conversations and generate contextually rich responses.

*  Dynamic Learning and Adaptation:
Implement a learning mechanism to allow the chatbot to adapt and improve over time based on user interactions. This could involve fine-tuning the model with user feedback or integrating online learning techniques.

* Multi-Turn Conversation Handling:
Enhance the chatbot's ability to handle multi-turn conversations more effectively. Implement context-awareness to maintain coherence and relevance across extended interactions.

*  User Authentication and Personalization:
Introduce user authentication to provide personalized responses. Store user preferences and history to tailor the chatbot's interactions based on individual user profiles.

*  Support for Rich Media:
Extend the chatbot's capabilities to handle and generate responses containing rich media such as images, videos, and documents. This can enhance the user experience and support a wider range of queries.

* Contextual Intent Recognition:
Improve intent recognition by considering contextual information from previous user interactions. This can help the chatbot better understand user queries and provide more accurate responses.

* Integration with External APIs:
Extend the chatbot's functionality by integrating with external APIs. This could include fetching real-time data, providing live updates, or performing specific actions beyond the current customized responses.

* Enhanced Error Handling:
Implement a robust error-handling mechanism to gracefully handle situations where the chatbot cannot provide a satisfactory response. This could involve offering alternative suggestions or guiding users to relevant resources.

* Advanced Security Measures:
Strengthen security measures by implementing end-to-end encryption for user interactions. Additionally, explore methods to identify and prevent potential security threats, ensuring the chatbot's reliability and trustworthiness.

* Multi-Language Support:
Enable the chatbot to understand and respond in multiple languages, expanding its reach to a global audience and accommodating users who prefer communicating in languages other than English.

* Conversational UX Design:
Collaborate with UX designers to enhance the conversational user interface, making interactions more intuitive and engaging. This includes refining the chatbot's tone, handling interruptions gracefully, and providing clear guidance to users.

* Continuous Model Training:
Implement a mechanism for continuous model training to keep the chatbot up-to-date with evolving language patterns and user expectations. Regularly update the model to adapt to new trends and information.




# Cloud infrastructure and cost comparison (AWS vs. Azure):
Developing a customized chatbot presented limitations for suggesting optimal cloud infrastructure and conducting a cost comparison between AWS and Azure. Rigorous testing on a real-time chatbot with more sophisticated capabilities is essential for accurate assessments. Key findings revealed scalability challenges and constrained user experience with the current approach. Future enhancements, including advanced NLP models and dynamic learning, are crucial for addressing limitations and forming the basis for a more informed cloud infrastructure decision.